In [1]:
from __future__ import annotations

import glob
import os
import time

import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from pettingzoo.test import api_test
from pettingzoo.utils.conversions import aec_to_parallel
from lib.briscola_env.briscola_env import BriscolaEnv

In [2]:
env = BriscolaEnv()
api_test(env, num_cycles=1000)


Starting API test
Passed API test


/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:242: UserWarning: Observation space for each agent probably should be gymnasium.spaces.box or gymnasium.spaces.discrete
  warnings.warn(
/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:140: UserWarning: Observation is not a NumPy array
  warnings.warn("Observation is not a NumPy array")
/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:661: UserWarning: Environment has not defined a render() method
  warnings.warn("Environment has not defined a render() method")


In [3]:
def make_env():
    env = BriscolaEnv()
    env = aec_to_parallel(env)
    return env

In [12]:
def train(
    steps: int = 10_000, seed: int | None = 0, **env_kwargs
):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = make_env()

    env.reset(seed=seed)

    print(f"Starting training on {str(env.metadata['name'])}.")

    env = ss.pettingzoo_env_to_vec_env_v1(env)
    env = ss.concat_vec_envs_v1(env, 8, num_cpus=1, base_class="stable_baselines3")

    # observation space is discrete so we use an MLP policy rather than CNN
    model = PPO(
        MlpPolicy,
        env,
        verbose=3,
        learning_rate=1e-3,
        batch_size=256,
    )
    model.learn(total_timesteps=steps)
    model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    print("Model has been saved.")
    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")

    env.close()

In [14]:
train()

Starting training on briscola.
Using cuda device


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'config' (most likely due to a circular import)